## [最適化におけるPython - Qiita](https://qiita.com/SaitoTsutomu/items/070ca9cb37c6b2b492f0)


+ 数理最適化問題をとくときのステップ
    1. モデラーを作成して数理モデルを用意する
    1. 数理モデルをソルバーに入力し、解を得る。
+ Pythonで使うモデラー PuLP
+ PuPL で扱える問題は混合整数最適化問題
    + 連続（実数）変数と整数変数を使って表現
    + 目的関数と制約条件が一次式




### 例題
```
材料AとBを合成して出来る化学製品XとYをできるだけたくさん作りたい。
Xを作るにはAを１ｋｇ、Bを３ｋｇ
Yを作るにはAを２ｋｇ、Bを１ｋｇ必要
XとYの売価は１ｋｇあたり１００円
いま、材料Aは１６ｋｇ、Bは１８ｋｇある
このときXとYの合計販売額が最大になるようにするには、XとYをどれだけ作成すればよいか？
```

item |　式
---| --- 
変数 | x, y >= 0 
目的変数 | 100x + 100y を最大化
制約条件 | x + 2y <= 16 <br> 3x + y <= 18


In [11]:
from pulp import * 
m = LpProblem(sense=LpMaximize) # 数理モデル 最小化問題のとき: 最小化問題のとき: m = LpPrblem()
x = LpVariable('x', lowBound=0) # 連続変数　lowbound=0：非負　
y = LpVariable('y', lowBound=0) # 連続変数　
# ちなみに、連続変数のリストを作るには 
# x = [LpVariable(i番目の変数名, lowBound=0) for i in range(n)] で、変数名は必ず一意

# まずは目的関数を追加　目的関数は m += 式
m += 100 * x + 100 * y

# 制約条件を追加 制約条件は m += 式 == 式 / m += 式 <= 式 / m += 式 >= 式 
m += x + 2*y <= 16
m += 3*x + y <= 18

m.solve()

print(value(x), value(y))


4.0 6.0


式の例は、

+ 和：lpSum(変数のリスト）
+ 内積： lpDot(係数リスト, 変数リスト) 

最大化、最小化のオプション
- LpProblem(sense=LpMaximize) のところは、デフォルトはLpMinimize、これは `LpMinimize = 1` でもあるので、m1 って書くんだね！
- sense=LpMaximize(LpMaximize = -1)





## 輸送最適化問題

+ 変数：倉庫群から工場群へ輸送個数を決めたい
+ 目的関数：輸送コストの最小化
    + コストテーブルに輸送個数をかけて足し合わせたものを最小化
+ 制約
    + 倉庫からの供給可能量以下
    + 工場の需要量以上

---
+ 倉庫から工場へのコスト

工場|F1|F2|F3|F4
---|---|---|---|---
W1|10|10|11|27
W2|18|21|12|14
W3|15|12|14|12

+ 各倉庫の供給可能最大値

W1|W2|W3
---|---|---
35|41|42

+ 各工場の需要最小値

F1|F2|F3|F4
---|---|---|---
28|29|31|25



In [27]:
import pandas as pd
import numpy as np
from itertools import product
from pulp import * 
np.random.seed(1)

# 倉庫は３つ、工場は４つ
supplies = [35,41,42]
demands = [28,29,31,25]

nw = len(supplies)
nf = len(demands)

pr = list(product(range(nw), range(nf)))

costs = [[10,10,11,27],[18,21,12,14],[15,12,14,12]]



In [28]:
# モデル作成
m1 = LpProblem()
v1 = {(i,j): LpVariable(f'v{i}_{j}', lowBound=0) for i,j in pr}


In [34]:
from pprint import pprint
pprint(v1, width=40)


{(0, 0): v0_0,
 (0, 1): v0_1,
 (0, 2): v0_2,
 (0, 3): v0_3,
 (1, 0): v1_0,
 (1, 1): v1_1,
 (1, 2): v1_2,
 (1, 3): v1_3,
 (2, 0): v2_0,
 (2, 1): v2_1,
 (2, 2): v2_2,
 (2, 3): v2_3}


.|j0|j1|j2|j3
---|---|---|---|---
i0|(0, 0): v0_0| (0, 1): v0_1| (0, 2): v0_2| (0, 3): v0_3
i1|(1, 0): v1_0| (1, 1): v1_1| (1, 2): v1_2| (1, 3): v1_3
i2|(2, 0): v2_0| (2, 1): v2_1| (2, 2): v2_2| (2, 3): v2_3


In [25]:
# 目的関数を追加 = コストテーブルに輸送個数をかけて足し合わせたものを最小化
m1 += lpSum([costs[i][j] * v1[i,j] for i, j in pr]) # コレを最小化


In [35]:
# 制約条件
for i in range(nw):
    m1 += lpSum(v1[i,j] for j in range(nf)) <= supplies[i]
    

In [37]:
for j in range(nf):
    m1 += lpSum(v1[i, j] for i in range(nw)) >= demands[j]

In [38]:
m1.solve()

1

In [49]:
# こんなんでました。
num_ships = pd.Series({k:value(v) for k, v in v1.items()}).unstack()

In [54]:
pd.DataFrame(costs) * num_ships 


,0,1,2,3
0,0.0,340.0,0.0,27.0
1,0.0,0.0,372.0,140.0
2,420.0,0.0,0.0,168.0


In [57]:
(pd.DataFrame(costs) * num_ships ).sum().sum()

1467.0

## Pandas で解くぞ

In [65]:
df_v = pd.DataFrame([(i, j) for i, j in pr], columns=["倉庫", "工場"])
df_v["コスト"] = pd.DataFrame(costs).values.flatten()
df_v.head()

,倉庫,工場,コスト
0,0,0,10
1,0,1,10
2,0,2,11
3,0,3,27
4,1,0,18


In [66]:
# モデル作成
m2 = LpProblem()


In [69]:
df_v["Var"] = [LpVariable(f'v{i}', lowBound=0) for i in df_v.index]
df_v.head()

,倉庫,工場,コスト,Var
0,0,0,10,v0
1,0,1,10,v1
2,0,2,11,v2
3,0,3,27,v3
4,1,0,18,v4


In [72]:
# 目的関数
m2 += lpDot(df_v["コスト"], df_v["Var"])

In [73]:
# 制約条件
for i, df in df_v.groupby("倉庫"):
    m2 += lpSum(df["Var"]) <= supplies[i]

for i, df in df_v.groupby("工場"):
    m2 += lpSum(df["Var"]) >= demands[i]
    

In [74]:
m2.solve()


1

In [77]:
df_v["num_of_ship"] = df_v["Var"].apply(value)

In [80]:
(df_v["コスト"] * df_v["num_of_ship"]).sum()

1296.0